ddl.py

In [1]:
import mysql.connector

conn = mysql.connector.connect(
    host='maglev.proxy.rlwy.net',
    port=41425,
    user='root',    
    password='zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW'
)
cursor = conn.cursor()

cursor.execute("DROP DATABASE IF EXISTS pos_management_db")
cursor.execute("CREATE DATABASE IF NOT EXISTS pos_management_db")
cursor.execute("USE pos_management_db")

table_statements = [

    """
    CREATE TABLE pos_system (
        pos_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        description TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """,

    """
    CREATE TABLE pos_feature (
        feature_id INT AUTO_INCREMENT PRIMARY KEY,
        feature_name VARCHAR(150) NOT NULL,
        description TEXT
    )
    """,

    """
    CREATE TABLE pricing (
        pricing_id INT AUTO_INCREMENT PRIMARY KEY,
        pos_id INT,
        plan_name VARCHAR(100) NOT NULL,
        price_per_month DECIMAL(10, 2) NOT NULL,
        features_summary TEXT,
        FOREIGN KEY (pos_id) REFERENCES pos_system(pos_id)
    )
    """,

    """
    CREATE TABLE store_client (
        store_id INT AUTO_INCREMENT PRIMARY KEY,
        store_name VARCHAR(150) NOT NULL,
        owner_name VARCHAR(100),
        email VARCHAR(100),
        phone VARCHAR(20),
        address VARCHAR(255),
        pricing_id INT,
        activated_at TIMESTAMP,
        FOREIGN KEY (pricing_id) REFERENCES pricing(pricing_id)
    )
    """,

    """
    CREATE TABLE pos_sale (
        sale_id INT AUTO_INCREMENT PRIMARY KEY,
        store_id INT,
        pos_id INT,
        pricing_id INT,
        amount DECIMAL(10,2),
        sale_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (store_id) REFERENCES store_client(store_id),
        FOREIGN KEY (pos_id) REFERENCES pos_system(pos_id),
        FOREIGN KEY (pricing_id) REFERENCES pricing(pricing_id)
    )
    """,

    """
    CREATE TABLE store_inventory (
        inventory_id INT AUTO_INCREMENT PRIMARY KEY,
        store_id INT,
        product_name VARCHAR(150),
        barcode VARCHAR(50),
        quantity INT NOT NULL DEFAULT 0,
        unit_price DECIMAL(10, 2) NOT NULL,
        FOREIGN KEY (store_id) REFERENCES store_client(store_id)
    )
    """,

    """
    CREATE TABLE store_transaction (
        transaction_id INT AUTO_INCREMENT PRIMARY KEY,
        store_id INT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (store_id) REFERENCES store_client(store_id)
    )
    """,

    """
    CREATE TABLE transaction_item (
        inventory_id INT,
        transaction_id INT,
        quantity INT NOT NULL,
        unit_price DECIMAL(10,2) NOT NULL,
        FOREIGN KEY (transaction_id) REFERENCES store_transaction(transaction_id),
        FOREIGN KEY (inventory_id) REFERENCES store_inventory(inventory_id)
    )
    """,

    """
    CREATE TABLE system_feature (
        pos_id INT,
        feature_id INT,
        FOREIGN KEY (pos_id) REFERENCES pos_system(pos_id),
        FOREIGN KEY (feature_id) REFERENCES pos_feature(feature_id)
    )
    """
]

for statement in table_statements:
    cursor.execute(statement)

print("Database and all tables created successfully.")

cursor.close()
conn.close()

Database and all tables created successfully.


dml.py

In [ ]:
from faker import Faker
import random
import mysql.connector

faker = Faker()

conn = mysql.connector.connect(
    host='maglev.proxy.rlwy.net',
    port=41425,
    user='root',    
    password='zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW',
    database='pos_management_db'
)

cursor = conn.cursor()

def get_ids(table_name, id_column):
    cursor.execute(f"SELECT {id_column} FROM {table_name}")
    return [row[0] for row in cursor.fetchall()]

for _ in range(100):
    name = faker.company()
    description = faker.text()
    cursor.execute("INSERT INTO pos_system (name, description) VALUES (%s, %s)", (name, description))

conn.commit()
print("Inserted 100 records into pos_system.")
pos_ids = get_ids("pos_system", "pos_id")

for _ in range(100):
    feature_name = faker.bs().capitalize()
    description = faker.text()
    cursor.execute("INSERT INTO pos_feature (feature_name, description) VALUES (%s, %s)", (feature_name, description))

conn.commit()
print("Inserted 100 records into pos_feature.")
feature_ids = get_ids("pos_feature", "feature_id")

for _ in range(100):
    pos_id = random.choice(pos_ids)
    plan_name = faker.word().capitalize() + " Plan"
    price_per_month = round(random.uniform(10, 200), 2)
    features_summary = faker.sentence()
    cursor.execute("""
        INSERT INTO pricing (pos_id, plan_name, price_per_month, features_summary)
        VALUES (%s, %s, %s, %s)
    """, (pos_id, plan_name, price_per_month, features_summary))

conn.commit()
print("Inserted 100 records into pricing.")
pricing_ids = get_ids("pricing", "pricing_id")

for _ in range(100):
    store_name = faker.company()
    owner_name = faker.name()
    email = faker.email()
    phone = faker.phone_number()[:20]
    address = faker.address().replace('\n', ', ')
    pricing_id = random.choice(pricing_ids)
    activated_at = faker.date_time_this_year()
    cursor.execute("""
        INSERT INTO store_client (store_name, owner_name, email, phone, address, pricing_id, activated_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (store_name, owner_name, email, phone, address, pricing_id, activated_at))

conn.commit()
print("Inserted 100 records into store_client.")
store_ids = get_ids("store_client", "store_id")

for _ in range(100):
    store_id = random.choice(store_ids)
    pos_id = random.choice(pos_ids)
    pricing_id = random.choice(pricing_ids)
    amount = round(random.uniform(100, 2000), 2)
    cursor.execute("""
        INSERT INTO pos_sale (store_id, pos_id, pricing_id, amount)
        VALUES (%s, %s, %s, %s)
    """, (store_id, pos_id, pricing_id, amount))

conn.commit()
print("Inserted 100 records into pos_sale.")

inventory_ids = []
for _ in range(100):
    store_id = random.choice(store_ids)
    product_name = faker.word().capitalize()
    barcode = faker.unique.ean(length=13)
    quantity = random.randint(1, 100)
    unit_price = round(random.uniform(1, 50), 2)
    cursor.execute("""
        INSERT INTO store_inventory (store_id, product_name, barcode, quantity, unit_price)
        VALUES (%s, %s, %s, %s, %s)
    """, (store_id, product_name, barcode, quantity, unit_price))

conn.commit()
print("Inserted 100 records into store_inventory.")
inventory_ids = get_ids("store_inventory", "inventory_id")

transaction_ids = []
for _ in range(100):
    store_id = random.choice(store_ids)
    cursor.execute("INSERT INTO store_transaction (store_id) VALUES (%s)", (store_id,))

conn.commit()
print("Inserted 100 records into store_transaction.")
transaction_ids = get_ids("store_transaction", "transaction_id")

for _ in range(100):
    inventory_id = random.choice(inventory_ids)
    transaction_id = random.choice(transaction_ids)
    quantity = random.randint(1, 10)
    unit_price = round(random.uniform(5, 100), 2)
    cursor.execute("""
        INSERT INTO transaction_item (inventory_id, transaction_id, quantity, unit_price)
        VALUES (%s, %s, %s, %s)
    """, (inventory_id, transaction_id, quantity, unit_price))

conn.commit()
print("Inserted 100 records into transaction_item.")

for _ in range(100):
    pos_id = random.choice(pos_ids)
    feature_id = random.choice(feature_ids)
    cursor.execute("""
        INSERT INTO system_feature (pos_id, feature_id)
        VALUES (%s, %s)
    """, (pos_id, feature_id))

conn.commit()
print("Inserted 100 records into system_feature.")

cursor.close()
conn.close()

print("Data insertion completed successfully.")


dql.py

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host='maglev.proxy.rlwy.net',
    port=41425,
    user='root',    
    password='zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW',
    database='pos_management_db'
)
cursor = conn.cursor()

queries = [
    ("Query 1: Store info with POS and pricing",
    """
    SELECT sc.store_name, sc.owner_name, sc.email,
           ps.name as pos_system_name, p.plan_name,
           p.price_per_month, sc.activated_at
    FROM store_client sc
    JOIN pricing p ON sc.pricing_id = p.pricing_id
    JOIN pos_system ps ON p.pos_id = ps.pos_id
    ORDER BY sc.activated_at DESC
    """),

    ("Query 2: Total revenue by POS system",
    """
    SELECT ps.name as pos_system_name,
           COUNT(ps_sale.sale_id) as total_sales,
           SUM(ps_sale.amount) as total_revenue,
           AVG(ps_sale.amount) as average_sale_amount
    FROM pos_sale ps_sale
    JOIN pos_system ps ON ps_sale.pos_id = ps.pos_id
    GROUP BY ps.pos_id, ps.name
    ORDER BY total_revenue DESC
    """),

    ("Query 3: Above-average inventory value stores",
    """
    SELECT sc.store_name, sc.owner_name,
           SUM(si.quantity * si.unit_price) as total_inventory_value
    FROM store_client sc
    JOIN store_inventory si ON sc.store_id = si.store_id
    GROUP BY sc.store_id, sc.store_name, sc.owner_name
    HAVING SUM(si.quantity * si.unit_price) > (
        SELECT AVG(store_total.total_value)
        FROM (
            SELECT SUM(quantity * unit_price) as total_value
            FROM store_inventory
            GROUP BY store_id
        ) as store_total
    )
    ORDER BY total_inventory_value DESC
    """),

    ("Query 4: Transaction details with product info",
    """
    SELECT st.transaction_id, sc.store_name, si.product_name,
           ti.quantity, ti.unit_price,
           (ti.quantity * ti.unit_price) as total_amount,
           st.created_at
    FROM store_transaction st
    JOIN store_client sc ON st.store_id = sc.store_id
    JOIN transaction_item ti ON st.transaction_id = ti.transaction_id
    JOIN store_inventory si ON ti.inventory_id = si.inventory_id
    WHERE st.created_at >= '2024-01-01'
    ORDER BY st.created_at DESC
    """),

    ("Query 5: Store sales rank",
    """
    SELECT sc.store_name, sc.owner_name,
           SUM(ps_sale.amount) as total_revenue,
           RANK() OVER (ORDER BY SUM(ps_sale.amount) DESC) as revenue_rank,
           DENSE_RANK() OVER (ORDER BY SUM(ps_sale.amount) DESC) as dense_revenue_rank
    FROM store_client sc
    LEFT JOIN pos_sale ps_sale ON sc.store_id = ps_sale.store_id
    GROUP BY sc.store_id, sc.store_name, sc.owner_name
    ORDER BY total_revenue DESC
    """),

    ("Query 6: POS systems with all features",
    """
    SELECT ps.name as pos_system_name, ps.description
    FROM pos_system ps
    WHERE NOT EXISTS (
        SELECT 1 
        FROM pos_feature pf
        WHERE NOT EXISTS (
            SELECT 1 
            FROM system_feature sf 
            WHERE sf.pos_id = ps.pos_id AND sf.feature_id = pf.feature_id
        )
    )
    """),

    ("Query 7: Monthly sales performance",
    """
    SELECT YEAR(st.created_at) as year,
           MONTH(st.created_at) as month,
           COUNT(DISTINCT st.store_id) as active_stores,
           COUNT(st.transaction_id) as total_transactions,
           SUM(ti.quantity * ti.unit_price) as total_sales,
           CASE 
               WHEN SUM(ti.quantity * ti.unit_price) > 1000 THEN 'High Performance'
               WHEN SUM(ti.quantity * ti.unit_price) > 500 THEN 'Medium Performance'
               ELSE 'Low Performance'
           END as performance_category
    FROM store_transaction st
    JOIN transaction_item ti ON st.transaction_id = ti.transaction_id
    GROUP BY YEAR(st.created_at), MONTH(st.created_at)
    ORDER BY year DESC, month DESC
    """),

    ("Query 8: Compare pricing plans within same POS system",
    """
    SELECT p1.plan_name as plan1_name,
           p1.price_per_month as plan1_price,
           p2.plan_name as plan2_name,
           p2.price_per_month as plan2_price,
           (p2.price_per_month - p1.price_per_month) as price_difference,
           ps.name as pos_system_name
    FROM pricing p1
    JOIN pricing p2 ON p1.pos_id = p2.pos_id AND p1.pricing_id < p2.pricing_id
    JOIN pos_system ps ON p1.pos_id = ps.pos_id
    ORDER BY price_difference DESC
    """),

    ("Query 9: Stores with high-value transactions",
    """
    SELECT sc.store_name, sc.owner_name,
           COUNT(st.transaction_id) as transaction_count,
           AVG(transaction_total.total_amount) as avg_transaction_value
    FROM store_client sc
    JOIN store_transaction st ON sc.store_id = st.store_id
    JOIN (
        SELECT transaction_id,
               SUM(quantity * unit_price) as total_amount
        FROM transaction_item
        GROUP BY transaction_id
    ) transaction_total ON st.transaction_id = transaction_total.transaction_id
    WHERE transaction_total.total_amount IN (
        SELECT SUM(quantity * unit_price)
        FROM transaction_item
        GROUP BY transaction_id
        HAVING SUM(quantity * unit_price) > 50
    )
    GROUP BY sc.store_id, sc.store_name, sc.owner_name
    ORDER BY avg_transaction_value DESC
    """),

    ("Query 10: Feature usage analysis",
    """
    SELECT pf.feature_name,
           COUNT(DISTINCT sf.pos_id) as pos_systems_with_feature,
           COUNT(DISTINCT ps_sale.store_id) as stores_using_feature,
           SUM(ps_sale.amount) as total_revenue_from_feature,
           ROUND(
               (COUNT(DISTINCT sf.pos_id) * 100.0 / (SELECT COUNT(*) FROM pos_system)), 2
           ) as feature_adoption_percentage
    FROM pos_feature pf
    LEFT JOIN system_feature sf ON pf.feature_id = sf.feature_id
    LEFT JOIN pos_sale ps_sale ON sf.pos_id = ps_sale.pos_id
    GROUP BY pf.feature_id, pf.feature_name
    ORDER BY feature_adoption_percentage DESC
    """),

    ("Query 11: Inventory turnover per store",
    """
    SELECT sc.store_name,
           COUNT(si.inventory_id) as total_products,
           SUM(si.quantity) as total_stock,
           SUM(si.quantity * si.unit_price) as inventory_value,
           COUNT(ti.transaction_id) as sales_transactions,
           CASE 
               WHEN COUNT(ti.transaction_id) > 0 THEN 
                   ROUND(SUM(si.quantity * si.unit_price) / COUNT(ti.transaction_id), 2)
               ELSE 0 
           END as inventory_per_transaction
    FROM store_client sc
    LEFT JOIN store_inventory si ON sc.store_id = si.store_id
    LEFT JOIN transaction_item ti ON si.inventory_id = ti.inventory_id
    GROUP BY sc.store_id, sc.store_name
    ORDER BY inventory_per_transaction DESC
    """),

    ("Query 12: Monthly growth trend",
    """
    SELECT YEAR(st.created_at) as year,
           MONTH(st.created_at) as month,
           COUNT(st.transaction_id) as transactions,
           SUM(ti.quantity * ti.unit_price) as revenue,
           LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at)) as prev_month_revenue,
           CASE 
               WHEN LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at)) > 0 THEN
                   ROUND(
                       ((SUM(ti.quantity * ti.unit_price) - LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at))) / 
                       LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at))) * 100, 2
                   )
               ELSE 0 
           END as growth_percentage
    FROM store_transaction st
    JOIN transaction_item ti ON st.transaction_id = ti.transaction_id
    GROUP BY YEAR(st.created_at), MONTH(st.created_at)
    ORDER BY year DESC, month DESC
    """)
]

title, sql = queries[0] # Change [0] -> to Test Other Queries

try:
    cursor.execute(sql)
    rows = cursor.fetchall()
    print(title)
    for row in rows:
        print(row)
    if not rows:
        print("[No results]")
except Exception as e:
    print(f"[Error running query]: {e}")

cursor.close()
conn.close()

index.py

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host='maglev.proxy.rlwy.net',
    port=41425,
    user='root',    
    password='zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW',
    database='pos_management_db'
)
cursor = conn.cursor()

index_queries = [
    # 1. Primary Key Indexes (Already Auto Created)

    # 2. Foreign Key Indexes
    "CREATE INDEX idx_pricing_pos_id ON pricing(pos_id)",
    "CREATE INDEX idx_store_client_pricing_id ON store_client(pricing_id)",
    "CREATE INDEX idx_pos_sale_store_id ON pos_sale(store_id)",
    "CREATE INDEX idx_pos_sale_pos_id ON pos_sale(pos_id)",
    "CREATE INDEX idx_pos_sale_pricing_id ON pos_sale(pricing_id)",
    "CREATE INDEX idx_store_inventory_store_id ON store_inventory(store_id)",
    "CREATE INDEX idx_store_transaction_store_id ON store_transaction(store_id)",
    "CREATE INDEX idx_transaction_item_transaction_id ON transaction_item(transaction_id)",
    "CREATE INDEX idx_transaction_item_inventory_id ON transaction_item(inventory_id)",
    "CREATE INDEX idx_system_feature_pos_id ON system_feature(pos_id)",
    "CREATE INDEX idx_system_feature_feature_id ON system_feature(feature_id)",

    # 3. Composite Indexes
    "CREATE INDEX idx_store_transaction_store_date ON store_transaction(store_id, created_at)",
    "CREATE INDEX idx_pos_sale_store_date ON pos_sale(store_id, sale_date)",
    "CREATE INDEX idx_store_inventory_store_product ON store_inventory(store_id, product_name)",
    "CREATE INDEX idx_transaction_item_inventory_quantity ON transaction_item(inventory_id, quantity)",

    # 4. WHERE clause optimization
    "CREATE INDEX idx_store_client_email ON store_client(email)",
    "CREATE INDEX idx_store_client_store_name ON store_client(store_name)",
    "CREATE INDEX idx_store_inventory_barcode ON store_inventory(barcode)",
    "CREATE INDEX idx_store_inventory_product_name ON store_inventory(product_name)",
    "CREATE INDEX idx_pos_system_name ON pos_system(name)",
    "CREATE INDEX idx_pos_feature_name ON pos_feature(feature_name)",

    # 5. ORDER BY optimization
    "CREATE INDEX idx_store_transaction_created_at ON store_transaction(created_at)",
    "CREATE INDEX idx_pos_sale_sale_date ON pos_sale(sale_date)",
    "CREATE INDEX idx_store_client_activated_at ON store_client(activated_at)",

    # 6. Aggregation queries
    "CREATE INDEX idx_store_inventory_store_price ON store_inventory(store_id, unit_price)",
    "CREATE INDEX idx_transaction_item_quantity_price ON transaction_item(quantity, unit_price)",

    # 7. Full-text search
    "CREATE FULLTEXT INDEX idx_store_inventory_product_search ON store_inventory(product_name, barcode)",
    "CREATE FULLTEXT INDEX idx_store_client_search ON store_client(store_name, owner_name, email)",

    # 8. Unique indexes
    "CREATE UNIQUE INDEX idx_store_client_email_unique ON store_client(email)",
    "CREATE UNIQUE INDEX idx_store_inventory_barcode_unique ON store_inventory(barcode)"
]

for query in index_queries:
    try:
        cursor.execute(query)
        print(f"Executed: {query}")
    except Exception as e:
        print(f"Error executing: {query}\n→ {e}")

conn.commit()
cursor.close()
conn.close()

print("\nAll indexing operations completed.")


dql_with_index.py

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host='maglev.proxy.rlwy.net',
    port=41425,
    user='root',    
    password='zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW',
    database='pos_management_db'
)
cursor = conn.cursor()

queries = [
    ("Query 1: Store info with POS and pricing",
    """
    SELECT sc.store_name, sc.owner_name, sc.email,
           ps.name as pos_system_name, p.plan_name,
           p.price_per_month, sc.activated_at
    FROM store_client sc
    JOIN pricing p ON sc.pricing_id = p.pricing_id
    JOIN pos_system ps ON p.pos_id = ps.pos_id
    ORDER BY sc.activated_at DESC
    """),

    ("Query 2: Total revenue by POS system",
    """
    SELECT ps.name as pos_system_name,
           COUNT(ps_sale.sale_id) as total_sales,
           SUM(ps_sale.amount) as total_revenue,
           AVG(ps_sale.amount) as average_sale_amount
    FROM pos_sale ps_sale
    JOIN pos_system ps ON ps_sale.pos_id = ps.pos_id
    GROUP BY ps.pos_id, ps.name
    ORDER BY total_revenue DESC
    """),

    ("Query 3: Above-average inventory value stores",
    """
    SELECT sc.store_name, sc.owner_name,
           SUM(si.quantity * si.unit_price) as total_inventory_value
    FROM store_client sc
    JOIN store_inventory si ON sc.store_id = si.store_id
    GROUP BY sc.store_id, sc.store_name, sc.owner_name
    HAVING SUM(si.quantity * si.unit_price) > (
        SELECT AVG(store_total.total_value)
        FROM (
            SELECT SUM(quantity * unit_price) as total_value
            FROM store_inventory
            GROUP BY store_id
        ) as store_total
    )
    ORDER BY total_inventory_value DESC
    """),

    ("Query 4: Transaction details with product info",
    """
    SELECT st.transaction_id, sc.store_name, si.product_name,
           ti.quantity, ti.unit_price,
           (ti.quantity * ti.unit_price) as total_amount,
           st.created_at
    FROM store_transaction st
    JOIN store_client sc ON st.store_id = sc.store_id
    JOIN transaction_item ti ON st.transaction_id = ti.transaction_id
    JOIN store_inventory si ON ti.inventory_id = si.inventory_id
    WHERE st.created_at >= '2024-01-01'
    ORDER BY st.created_at DESC
    """),

    ("Query 5: Store sales rank",
    """
    SELECT sc.store_name, sc.owner_name,
           SUM(ps_sale.amount) as total_revenue,
           RANK() OVER (ORDER BY SUM(ps_sale.amount) DESC) as revenue_rank,
           DENSE_RANK() OVER (ORDER BY SUM(ps_sale.amount) DESC) as dense_revenue_rank
    FROM store_client sc
    LEFT JOIN pos_sale ps_sale ON sc.store_id = ps_sale.store_id
    GROUP BY sc.store_id, sc.store_name, sc.owner_name
    ORDER BY total_revenue DESC
    """),

    ("Query 6: POS systems with all features",
    """
    SELECT ps.name as pos_system_name, ps.description
    FROM pos_system ps
    WHERE NOT EXISTS (
        SELECT 1 
        FROM pos_feature pf
        WHERE NOT EXISTS (
            SELECT 1 
            FROM system_feature sf 
            WHERE sf.pos_id = ps.pos_id AND sf.feature_id = pf.feature_id
        )
    )
    """),

    ("Query 7: Monthly sales performance",
    """
    SELECT YEAR(st.created_at) as year,
           MONTH(st.created_at) as month,
           COUNT(DISTINCT st.store_id) as active_stores,
           COUNT(st.transaction_id) as total_transactions,
           SUM(ti.quantity * ti.unit_price) as total_sales,
           CASE 
               WHEN SUM(ti.quantity * ti.unit_price) > 1000 THEN 'High Performance'
               WHEN SUM(ti.quantity * ti.unit_price) > 500 THEN 'Medium Performance'
               ELSE 'Low Performance'
           END as performance_category
    FROM store_transaction st
    JOIN transaction_item ti ON st.transaction_id = ti.transaction_id
    GROUP BY YEAR(st.created_at), MONTH(st.created_at)
    ORDER BY year DESC, month DESC
    """),

    ("Query 8: Compare pricing plans within same POS system",
    """
    SELECT p1.plan_name as plan1_name,
           p1.price_per_month as plan1_price,
           p2.plan_name as plan2_name,
           p2.price_per_month as plan2_price,
           (p2.price_per_month - p1.price_per_month) as price_difference,
           ps.name as pos_system_name
    FROM pricing p1
    JOIN pricing p2 ON p1.pos_id = p2.pos_id AND p1.pricing_id < p2.pricing_id
    JOIN pos_system ps ON p1.pos_id = ps.pos_id
    ORDER BY price_difference DESC
    """),

    ("Query 9: Stores with high-value transactions",
    """
    SELECT sc.store_name, sc.owner_name,
           COUNT(st.transaction_id) as transaction_count,
           AVG(transaction_total.total_amount) as avg_transaction_value
    FROM store_client sc
    JOIN store_transaction st ON sc.store_id = st.store_id
    JOIN (
        SELECT transaction_id,
               SUM(quantity * unit_price) as total_amount
        FROM transaction_item
        GROUP BY transaction_id
    ) transaction_total ON st.transaction_id = transaction_total.transaction_id
    WHERE transaction_total.total_amount IN (
        SELECT SUM(quantity * unit_price)
        FROM transaction_item
        GROUP BY transaction_id
        HAVING SUM(quantity * unit_price) > 50
    )
    GROUP BY sc.store_id, sc.store_name, sc.owner_name
    ORDER BY avg_transaction_value DESC
    """),

    ("Query 10: Feature usage analysis",
    """
    SELECT pf.feature_name,
           COUNT(DISTINCT sf.pos_id) as pos_systems_with_feature,
           COUNT(DISTINCT ps_sale.store_id) as stores_using_feature,
           SUM(ps_sale.amount) as total_revenue_from_feature,
           ROUND(
               (COUNT(DISTINCT sf.pos_id) * 100.0 / (SELECT COUNT(*) FROM pos_system)), 2
           ) as feature_adoption_percentage
    FROM pos_feature pf
    LEFT JOIN system_feature sf ON pf.feature_id = sf.feature_id
    LEFT JOIN pos_sale ps_sale ON sf.pos_id = ps_sale.pos_id
    GROUP BY pf.feature_id, pf.feature_name
    ORDER BY feature_adoption_percentage DESC
    """),

    ("Query 11: Inventory turnover per store",
    """
    SELECT sc.store_name,
           COUNT(si.inventory_id) as total_products,
           SUM(si.quantity) as total_stock,
           SUM(si.quantity * si.unit_price) as inventory_value,
           COUNT(ti.transaction_id) as sales_transactions,
           CASE 
               WHEN COUNT(ti.transaction_id) > 0 THEN 
                   ROUND(SUM(si.quantity * si.unit_price) / COUNT(ti.transaction_id), 2)
               ELSE 0 
           END as inventory_per_transaction
    FROM store_client sc
    LEFT JOIN store_inventory si ON sc.store_id = si.store_id
    LEFT JOIN transaction_item ti ON si.inventory_id = ti.inventory_id
    GROUP BY sc.store_id, sc.store_name
    ORDER BY inventory_per_transaction DESC
    """),

    ("Query 12: Monthly growth trend",
    """
    SELECT YEAR(st.created_at) as year,
           MONTH(st.created_at) as month,
           COUNT(st.transaction_id) as transactions,
           SUM(ti.quantity * ti.unit_price) as revenue,
           LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at)) as prev_month_revenue,
           CASE 
               WHEN LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at)) > 0 THEN
                   ROUND(
                       ((SUM(ti.quantity * ti.unit_price) - LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at))) / 
                       LAG(SUM(ti.quantity * ti.unit_price)) OVER (ORDER BY YEAR(st.created_at), MONTH(st.created_at))) * 100, 2
                   )
               ELSE 0 
           END as growth_percentage
    FROM store_transaction st
    JOIN transaction_item ti ON st.transaction_id = ti.transaction_id
    GROUP BY YEAR(st.created_at), MONTH(st.created_at)
    ORDER BY year DESC, month DESC
    """)
]

title, sql = queries[0] # Change [0] -> to Test Other Queries

try:
    cursor.execute(sql)
    rows = cursor.fetchall()
    print(title)
    for row in rows:
        print(row)
    if not rows:
        print("[No results]")
except Exception as e:
    print(f"[Error running query]: {e}")

cursor.close()
conn.close()

Additional.note